# Ai Model
This folder consist of SVM model to classify rice species. 
### Import all required packages

In [24]:
import pandas as pd 
import os 

### Conversion function 
Convert given excel dataset to csv for pandas dataframe.

In [25]:
def convert_do_csv(filePath):
    data = pd.read_excel(filePath)
    newPath = os.path.join(os.path.dirname(filePath), 'output.csv')
    data.to_csv(newPath, index=False)
    print("Conversion successful")

In [26]:
PATH = f"{os.path.abspath(os.path.join(os.getcwd(), os.pardir))}"
excelPath = os.path.join(PATH, "Rice_Dataset_Commeo_and_Osmancik/Rice_Cammeo_Osmancik.xlsx")

# convert_do_csv(filePath)

### SVM Model 
Import required packages for svm

In [27]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score

In [36]:
csv_path = os.path.join(PATH, "Rice_Dataset_Commeo_and_Osmancik/output.csv")

# Get data from csv file and split into feature matrix and target vector
rice_df = pd.read_csv(csv_path, index_col=0)
X = rice_df.drop(columns=["Class"])
Y = rice_df["Class"]

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svc', SVC())
])

param_grid = {
    'svc__kernel': ['linear', 'rbf', 'poly'],
    'svc__C': [0.1, 1, 10, 100],
    'svc__gamma': ['scale', 'auto', 0.001, 0.01, 0.1, 1]
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

grid_search = GridSearchCV(pipeline, param_grid, cv=cv, scoring='accuracy', n_jobs=-1)

grid_search.fit(X, Y)

print(f"Best Hyperparameters: {grid_search.best_params_}")

print(f"Best Cross-Validation Accuracy: {grid_search.best_score_:.2f}")

best_model = grid_search.best_estimator_

y_pred = best_model.predict(X)

print(classification_report(Y, y_pred))

Best Hyperparameters: {'svc__C': 10, 'svc__gamma': 'scale', 'svc__kernel': 'linear'}
Best Cross-Validation Accuracy: 0.93
              precision    recall  f1-score   support

      Cammeo       0.92      0.92      0.92      1630
    Osmancik       0.94      0.94      0.94      2180

    accuracy                           0.93      3810
   macro avg       0.93      0.93      0.93      3810
weighted avg       0.93      0.93      0.93      3810

